<div class="alert alert-info">
    
<b>Reference</b>
    
This tutorial is based on materials from the Keras Documentation:
* [Structured data classification from scratch](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)
    
</div>


<div class="alert alert-warning">

<b>Warning</b>
    
By design asyncio does not allow nested event loops. Jupyter is using Tornado which already starts an event loop. Therefore the following patch is required to run this tutorial.
    
This tutorial should be run with `tensorFlow>=2.6`.
    
</div>

In [1]:
!pip install nest_asyncio

import nest_asyncio
nest_asyncio.apply()

<div class="alert alert-info">
    
<b>Note</b>
    
The following can be used to avoid the logging of Tensorflow *DEBUG*, *INFO* and *WARNING* statements.
    
</div>

In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = str(3)

## Imports

<div class="alert alert-block alert-danger">
    
<b>Danger</b> 

The following cell contains Tensorflow import `import tensorflow as tf`. It is important to follow this strategy instead of `from tensorflow.keras.layers import ...` to avoid non-serializable data, creating crashed during the search. For example, the original Keras tutorial was using the following set of imports which was creating a serialization error in our use case.
    
```python
from tensorflow import keras
from tensorflow.keras import layers
...
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
```
    
</div>

In [3]:
import ray
import pandas as pd
import tensorflow as tf

<div class="alert alert-info">
    
<b>Note</b>
    
The following can be used to detect if **GPU** devices are available on the current host.
    
</div>

In [4]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == "GPU"]

n_gpus = len(get_available_gpus())
is_gpu_available = n_gpus > 0

if is_gpu_available:
    print(f"{n_gpus} GPU{'s are' if n_gpus > 1 else ' is'} available.")
else:
    print("No GPU available")

1 GPU is available.


### The dataset (from Keras.io)

The [dataset](https://archive.ics.uci.edu/ml/datasets/heart+Disease) is provided by the
Cleveland Clinic Foundation for Heart Disease.
It's a CSV file with 303 rows. Each row contains information about a patient (a
**sample**), and each column describes an attribute of the patient (a **feature**). We
use the features to predict whether a patient has a heart disease (**binary
classification**).

Here's the description of each feature:

Column| Description| Feature Type
------------|--------------------|----------------------
Age | Age in years | Numerical
Sex | (1 = male; 0 = female) | Categorical
CP | Chest pain type (0, 1, 2, 3, 4) | Categorical
Trestbpd | Resting blood pressure (in mm Hg on admission) | Numerical
Chol | Serum cholesterol in mg/dl | Numerical
FBS | fasting blood sugar in 120 mg/dl (1 = true; 0 = false) | Categorical
RestECG | Resting electrocardiogram results (0, 1, 2) | Categorical
Thalach | Maximum heart rate achieved | Numerical
Exang | Exercise induced angina (1 = yes; 0 = no) | Categorical
Oldpeak | ST depression induced by exercise relative to rest | Numerical
Slope | Slope of the peak exercise ST segment | Numerical
CA | Number of major vessels (0-3) colored by fluoroscopy | Both numerical & categorical
Thal | 3 = normal; 6 = fixed defect; 7 = reversible defect | Categorical
Target | Diagnosis of heart disease (1 = true; 0 = false) | Target

In [5]:
def load_data():
    file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
    dataframe = pd.read_csv(file_url)

    val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
    train_dataframe = dataframe.drop(val_dataframe.index)

    return train_dataframe, val_dataframe


def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

## Preprocessing & encoding of features

The next cells use `tf.keras.layers.Normalization()` to apply standard scaling on the features. Then, the `tf.keras.layers.StringLookup` and `tf.keras.layers.IntegerLookup` are used to encode categorical variables.

In [6]:
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = tf.keras.layers.Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = (
        tf.keras.layers.StringLookup if is_string else tf.keras.layers.IntegerLookup
    )
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

## Define the run-function

In [7]:
def run(config: dict):
    train_dataframe, val_dataframe = load_data()

    train_ds = dataframe_to_dataset(train_dataframe)
    val_ds = dataframe_to_dataset(val_dataframe)

    train_ds = train_ds.batch(config["batch_size"])
    val_ds = val_ds.batch(config["batch_size"])

    # Categorical features encoded as integers
    sex = tf.keras.Input(shape=(1,), name="sex", dtype="int64")
    cp = tf.keras.Input(shape=(1,), name="cp", dtype="int64")
    fbs = tf.keras.Input(shape=(1,), name="fbs", dtype="int64")
    restecg = tf.keras.Input(shape=(1,), name="restecg", dtype="int64")
    exang = tf.keras.Input(shape=(1,), name="exang", dtype="int64")
    ca = tf.keras.Input(shape=(1,), name="ca", dtype="int64")

    # Categorical feature encoded as string
    thal = tf.keras.Input(shape=(1,), name="thal", dtype="string")

    # Numerical features
    age = tf.keras.Input(shape=(1,), name="age")
    trestbps = tf.keras.Input(shape=(1,), name="trestbps")
    chol = tf.keras.Input(shape=(1,), name="chol")
    thalach = tf.keras.Input(shape=(1,), name="thalach")
    oldpeak = tf.keras.Input(shape=(1,), name="oldpeak")
    slope = tf.keras.Input(shape=(1,), name="slope")

    all_inputs = [
        sex,
        cp,
        fbs,
        restecg,
        exang,
        ca,
        thal,
        age,
        trestbps,
        chol,
        thalach,
        oldpeak,
        slope,
    ]

    # Integer categorical features
    sex_encoded = encode_categorical_feature(sex, "sex", train_ds, False)
    cp_encoded = encode_categorical_feature(cp, "cp", train_ds, False)
    fbs_encoded = encode_categorical_feature(fbs, "fbs", train_ds, False)
    restecg_encoded = encode_categorical_feature(restecg, "restecg", train_ds, False)
    exang_encoded = encode_categorical_feature(exang, "exang", train_ds, False)
    ca_encoded = encode_categorical_feature(ca, "ca", train_ds, False)

    # String categorical features
    thal_encoded = encode_categorical_feature(thal, "thal", train_ds, True)

    # Numerical features
    age_encoded = encode_numerical_feature(age, "age", train_ds)
    trestbps_encoded = encode_numerical_feature(trestbps, "trestbps", train_ds)
    chol_encoded = encode_numerical_feature(chol, "chol", train_ds)
    thalach_encoded = encode_numerical_feature(thalach, "thalach", train_ds)
    oldpeak_encoded = encode_numerical_feature(oldpeak, "oldpeak", train_ds)
    slope_encoded = encode_numerical_feature(slope, "slope", train_ds)

    all_features = tf.keras.layers.concatenate(
        [
            sex_encoded,
            cp_encoded,
            fbs_encoded,
            restecg_encoded,
            exang_encoded,
            slope_encoded,
            ca_encoded,
            thal_encoded,
            age_encoded,
            trestbps_encoded,
            chol_encoded,
            thalach_encoded,
            oldpeak_encoded,
        ]
    )
    x = tf.keras.layers.Dense(config["units"], activation=config["activation"])(
        all_features
    )
    x = tf.keras.layers.Dropout(config["dropout_rate"])(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(all_inputs, output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])
    model.compile(optimizer, "binary_crossentropy", metrics=["accuracy"])

    history = model.fit(
        train_ds, epochs=config["num_epochs"], validation_data=val_ds, verbose=0
    )

    return history.history["val_accuracy"][-1]

<div class="alert alert-success">
    
<b>Important</b>
    
The objective maximised by DeepHyper is the scalar value returned by the `run`-function. In this tutorial it corresponds to the validation accuracy of the last epoch of training which we retrieve in the `History` object returned by the `model.fit(...)` call.
    
```python
...
history = model.fit(
    train_ds, epochs=config["num_epochs"], validation_data=val_ds, verbose=0
)

return history.history["val_accuracy"][-1]
...
```

Using an objective like `max(history.history["val_accuracy"])` can have undesired effect such as training curves passing by a local maximum and then dropping which will not generate a model in capacity of ingesting well more data in the future.
    
</div>



## Evaluate a default configuration

In [8]:
default_config = {
    "units": 32,
    "activation": "relu",
    "dropout_rate": 0.5,
    "num_epochs": 50,
    "batch_size": 32,
    "learning_rate": 1e-3,
}

if is_gpu_available:
    
    ray.init(num_cpus=n_gpus, num_gpus=n_gpus)
    
    run_default = ray.remote(num_cpus=1, num_gpus=1)(run)
    
    objective_default = ray.get(run_default.remote(default_config))
else:
    run_default = run
    objective_default = run_default(default_config)
    
print(f"Accuracy Default Configuration:  {objective_default:.3f}")

2021-09-22 12:18:17,513	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265
2021-09-22 12:18:17,515	WARNING services.py:1749 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 8589934592 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68a01b51f0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68a01b51f0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_n

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68a01b5280> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68a01b5280>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68a01c49d0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68a01c49d0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68a01c4940> and will run it as-is.
(pid=247

Accuracy Default Configuration:  0.803


## Define the Hyperparameter optimization problem

In [9]:
ACTIVATIONS = [
    "elu",
    "gelu",
    "hard_sigmoid",
    "linear",
    "relu",
    "selu",
    "sigmoid",
    "softplus",
    "softsign",
    "swish",
    "tanh",
]

In [10]:
from deephyper.problem import HpProblem

problem = HpProblem()
problem.add_hyperparameter((8, 128), "units")
problem.add_hyperparameter(ACTIVATIONS, "activation")
problem.add_hyperparameter((0.0, 0.6), "dropout_rate")
problem.add_hyperparameter((10, 100), "num_epochs")
problem.add_hyperparameter((8, 256, "log-uniform"), "batch_size")
problem.add_hyperparameter((1e-5, 1e-2, "log-uniform"), "learning_rate")

problem

Configuration space object:
  Hyperparameters:
    activation, Type: Categorical, Choices: {elu, gelu, hard_sigmoid, linear, relu, selu, sigmoid, softplus, softsign, swish, tanh}, Default: elu
    batch_size, Type: UniformInteger, Range: [8, 256], Default: 45, on log-scale
    dropout_rate, Type: UniformFloat, Range: [0.0, 0.6], Default: 0.3
    learning_rate, Type: UniformFloat, Range: [1e-05, 0.01], Default: 0.0003162278, on log-scale
    num_epochs, Type: UniformInteger, Range: [10, 100], Default: 55
    units, Type: UniformInteger, Range: [8, 128], Default: 68

## Define the evaluator object

In [11]:
from deephyper.evaluator.evaluate import Evaluator

# Default arguments for Ray: 1 worker and 1 worker per evaluation
method_kwargs = {
    "num_cpus": 1, 
    "num_cpus_per_task": 1
}

# If GPU devices are detected then it will create 'n_gpus' workers
# and use 1 worker for each evaluation
if is_gpu_available:
    method_kwargs["num_cpus"] = n_gpus
    method_kwargs["num_gpus"] = n_gpus
    method_kwargs["num_cpus_per_task"] = 1
    method_kwargs["num_gpus_per_task"] = 1
    
print("Method Arguments: ", method_kwargs)

evaluator = Evaluator.create(
    run, 
    method="ray", 
    method_kwargs=method_kwargs
)

print("Evaluator's number of workers: ", evaluator.num_workers)

Method Arguments:  {'num_cpus': 1, 'num_cpus_per_task': 1, 'num_gpus': 1, 'num_gpus_per_task': 1}
Evaluator's number of workers:  1


## Define and run the asynchronous model-based search

In [12]:
from deephyper.search.hps import AMBS

search = AMBS(problem, evaluator)

In [13]:
results = search.search(max_evals=10)

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68587acb80> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68587acb80>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68587ace50> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68587ace50>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68587acee0> and will run it as-is

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68587ac790> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68587ac790>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68587acee0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68587acee0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68587ace50> and will run it as-is.
(pid=247

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68587acee0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68587acee0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f6858294e50> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f6858294e50>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f6858294430> and will run it as-is.
(pid=247

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f6858294040> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f6858294040>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f685850a160> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f685850a160>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f685850a1f0> and will run it as-is.


(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68a01d29d0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68a01d29d0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68782890d0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68782890d0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:5 out of the last 14 calls to <function PreprocessingLayer.make_adapt_function.<locals>.adapt_step at 0x7f68782c00d0> trigger

...

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68384b93a0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68384b93a0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68384b98b0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68384b98b0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68384b9430> and will run it as-is

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68384b9430> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68384b9430>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68384b98b0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68384b98b0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f6878105310> and will run it as-is.
(pid=247

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68a01d2b80> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68a01d2b80>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68782c0ca0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_numerical_feature.<locals>.<lambda> at 0x7f68782c0ca0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f68a00dcd30> and will run it as-is.
(pid=247

(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f6858151e50> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f6858151e50>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68584c3dc0> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68584c3dc0>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7f685851ef70> and will run it as-is.


(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68781d2700> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68781d2700>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68587d6160> and will run it as-is.
(pid=2476335) Cause: could not parse the source code of <function encode_categorical_feature.<locals>.<lambda> at 0x7f68587d6160>: no matching AST found
(pid=2476335) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(pid=2476335) WARNING:tensorflow:AutoGraph could not transform <function encode_categorical_feature.<locals>.<lambda> at 0x7f68587d6310> and will run it as-is

In [14]:
results

,activation,batch_size,dropout_rate,learning_rate,num_epochs,units,id,objective,elapsed_sec,duration
0,linear,17,0.311632,0.000676,71,97,1,0.786885,8.032742,7.282537
1,gelu,28,0.221418,0.000016,60,68,2,0.721311,13.982644,5.657617
2,linear,72,0.136193,0.000573,70,126,3,0.803279,18.642096,4.459822
3,gelu,98,0.394199,0.005590,65,118,4,0.786885,23.278052,4.436977
4,selu,161,0.240164,0.000459,41,128,5,0.868852,26.683581,3.195485
5,elu,168,0.396514,0.000036,10,128,6,0.475410,29.111158,2.224259
6,selu,34,0.064309,0.000546,13,114,7,0.836066,32.095055,2.782411
7,selu,48,0.179944,0.000104,12,128,8,0.754098,34.739235,2.444146
8,selu,187,0.370754,0.000257,10,12,9,0.655738,37.315197,2.376034
9,selu,153,0.589909,0.001054,11,94,10,0.819672,39.834914,2.320186


In [15]:
i_max = results.objective.argmax()
best_config = results.iloc[results.objective.argmax()][:-3].to_dict()

print(f"The default configuration has an accuracy of {objective_default:.3f}. " 
      f"The best configuration found by DeepHyper has an accuracy {results['objective'].iloc[i_max]:.3f}, " 
      f"trained in {results['duration'].iloc[i_max]:.2f} secondes and "
      f"finished after {results['elapsed_sec'].iloc[i_max]:.2f} secondes of search.")



best_config

The default configuration has an accuracy of 0.803. The best configuration found by DeepHyper has an accuracy 0.869, trained in 3.20 secondes and finished after 26.68 secondes of search.


{'activation': 'selu',
 'batch_size': 161,
 'dropout_rate': 0.2401638933318062,
 'learning_rate': 0.0004585594439639,
 'num_epochs': 41,
 'units': 128,
 'id': 5}